In [ ]:
import sys
import os
from collections import defaultdict
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
from glmpca import glmpca
from itertools import combinations
import torch
import scipy
from scipy.stats import pearsonr
from scipy import stats
from scipy.interpolate import UnivariateSpline
import gseapy as gp

import sys
from importlib import reload

import gaston
from gaston import neural_net,cluster_plotting, dp_related, segmented_fit, restrict_spots, model_selection
from gaston import binning_and_plotting, isodepth_scaling, run_slurm_scripts, parse_adata
from gaston import spatial_gene_classification, plot_cell_types, filter_genes, process_NN_output

import seaborn as sns
import math

## Import Data

In [ ]:
file_path='----/----_norm.h5ad' #replace with normalized h5ad file
adata=sc.read_h5ad(f'{file_path}')
HIF=pd.read_csv(f'public_data/HIF-1_signaling_pathway.csv')
EMT=pd.read_csv(f'public_data/EMT.csv')

#remove DEPRECATED genes
deprecated_genes = adata.var_names[adata.var_names.str.startswith('DEPRECATED')]
adata = adata[:, ~adata.var_names.isin(deprecated_genes)].copy()

counts_mat=np.array(adata.to_df())
coords_mat=adata.obsm['spatial']
coords_mat=np.fliplr(coords_mat)
gene_labels=np.array(adata.var.index)

use_RGB=False
# save matrices
np.save('test_data/counts_mat.npy', counts_mat)
np.save('test_data/coords_mat.npy', coords_mat)
np.save('test_data/gene_labels.npy', gene_labels)

In [ ]:
# GLM-PCA parameters
num_dims=5
penalty=20 # may need to increase if this is too small

# CHANGE THESE PARAMETERS TO REDUCE RUNTIME
num_iters=30
eps=1e-4
num_genes=10000

counts_mat_glmpca=counts_mat[:,np.argsort(np.sum(counts_mat, axis=0))[-num_genes:]]
glmpca_res=glmpca.glmpca(counts_mat_glmpca.T, num_dims, fam="poi", penalty=penalty, verbose=True,
                        ctl = {"maxIter":num_iters, "eps":eps, "optimizeTheta":True})
A = glmpca_res['factors'] # should be of size N x num_dims, where each column is a PC

if use_RGB:
    A=np.hstack((A,rgb_mean)) # attach to RGB mean
np.save('test_data/glmpca.npy', A)

In [ ]:
# visualize top GLM-PCs and RGB mean
rotated_coords=dp_related.rotate_by_theta(coords_mat, -np.pi/2)
R=1
C=5
fig,axs=plt.subplots(R,C,figsize=(24,4))
for r in range(R):
    for c in range(C):
        i=r*C+c
        axs[c].scatter(rotated_coords[:,0], rotated_coords[:,1], c=A[:,i],cmap='Reds',s=3)
        axs[c].set_title(f'GLM-PC{i}')

In [ ]:
# LOAD DATA generated above
path_to_glmpca='test_data/glmpca.npy'
path_to_coords='test_data/coords_mat.npy'

# GASTON NN parameters
isodepth_arch=[20,20] # architecture (two hidden layers of size 20) for isodepth neural network d(x,y)
expression_arch=[20,20] # architecture (two hidden layers of size 20) for 1-D expression function
epochs = 10000 # number of epochs to train NN
checkpoint = 500 # save model after number of epochs = multiple of checkpoint
optimizer = "adam"
num_restarts=30 # number of initializations

output_dir='test_outputs' # folder to save model runs

# REPLACE with your own conda environment name and path
conda_environment='gaston-package'
path_to_conda_folder='/mullib/miniconda3/bin/activate'

run_slurm_scripts.train_NN_parallel(path_to_coords, path_to_glmpca, isodepth_arch, expression_arch, 
                      output_dir, conda_environment, path_to_conda_folder,
                      epochs=epochs, checkpoint=checkpoint, 
                      num_seeds=num_restarts)

In [ ]:
gaston_model, A, S= process_NN_output.process_files('test_outputs') # model trained above
#gaston_model, A, S= process_NN_output.process_files('results/HH/02-S10-36130-1D/') 
    
# May need to re-load counts_mat, coords_mat, and gene_labels
counts_mat=np.load('test_data/counts_mat.npy',allow_pickle=True)
coords_mat=np.load('test_data/coords_mat.npy',allow_pickle=True)
gene_labels=np.load('test_data/gene_labels.npy',allow_pickle=True)

In [ ]:
model_selection.plot_ll_curve(gaston_model, A, S, max_domain_num=8, start_from=2)

In [ ]:
num_layers=4 # CHANGE FOR YOUR APPLICATION: use number of layers from above!
gaston_isodepth, gaston_labels=dp_related.get_isodepth_labels(gaston_model,A,S,num_layers)

# DATASET-SPECIFIC: so domains are ordered with tumor being last
gaston_isodepth= np.max(gaston_isodepth) -1 * gaston_isodepth
gaston_labels=(num_layers-1)-gaston_labels

In [ ]:
show_streamlines=True
rotate = np.radians(-90) # rotate coordinates by -90
arrowsize=2

cluster_plotting.plot_isodepth(gaston_isodepth, S, gaston_model, figsize=(8,6), streamlines=show_streamlines, 
                               rotate=rotate,arrowsize=arrowsize, neg_gradient=True)

In [ ]:
domain_colors=colors=['#16537e', '#93c47d', '#e99445', '#8ecae6','#6c4747']
domain_colors=colors=['#8ecae6','#e99445','#93c47d', '#16537e','#6c4747']

cluster_plotting.plot_clusters(gaston_labels, S, figsize=(6.7,6.5), 
                               colors=domain_colors, s=20, lgd=False, 
                               show_boundary=True, gaston_isodepth=gaston_isodepth, boundary_lw=5, rotate=rotate)

Restrict Spots

In [ ]:
# This is the range we used for reproducing figure papers.
isodepth_min=0
isodepth_max=8

cluster_plotting.plot_clusters_restrict(gaston_labels, S, gaston_isodepth, 
                                        isodepth_min=isodepth_min, isodepth_max=isodepth_max, figsize=(5,5), 
                                        colors=domain_colors, s=2, lgd=False, rotate=rotate)

In [ ]:
# Optional: adjust isodepth for physical distance
adjust_physical=False
scale_factor=100 # since distance of 1 = 100 microns for 10x Visium

# Optional: plot isodepth for green spots
plot_isodepth=False

# plotting parameters
show_streamlines=True
rotate=np.radians(-90)
arrowsize=1


counts_mat_restrict, coords_mat_restrict, gaston_isodepth_restrict, gaston_labels_restrict, S_restrict=restrict_spots.restrict_spots(
                                                             counts_mat, coords_mat, S, gaston_isodepth, gaston_labels, 
                                                             isodepth_min=isodepth_min, isodepth_max=isodepth_max, 
                                                             adjust_physical=adjust_physical, scale_factor=scale_factor,
                                                             plot_isodepth=plot_isodepth, show_streamlines=show_streamlines, 
                                                             gaston_model=gaston_model, rotate=rotate, figsize=(6,3), 
                                                             arrowsize=arrowsize, 
                                                             neg_gradient=True) # since we reversed gradient direction earlier

In [ ]:
df=pd.DataFrame(data=counts_mat_restrict,columns=gene_labels)
df2=df.assign(isodepth=gaston_isodepth_restrict)
correlation_list=[]
p_value_list=[]

for gene_name in df.columns:
    correlation,p_value=pearsonr(df[gene_name],df2['isodepth'])
    correlation_list.append(correlation)
    p_value_list.append(p_value)   

In [ ]:
correlation_matrix=[df.columns,correlation_list,p_value_list]
df4=pd.DataFrame(correlation_matrix)
new_header = df4.iloc[0] 
df4 = df4[1:] 
df4.columns = new_header
df4 = df4.reset_index(drop=True)
df4.index = ['correlation','p_value']
for i in df4.columns:
    if np.isnan(df4.loc['correlation',i]):
        del df4[i]
    if np.isnan(df4.loc['p_value',i]):
        del df4[i]
    if np.abs(df4.loc['correlation',i])<0.05:
        del df4[i]
df4=df4.sort_values(ascending=False,by = 'correlation', axis = 1) 
#df5 = df4.loc[:, df4.loc['p_value'] >= 0.00001]
#scipy.stats.false_discovery_control(p_values, method='bh')
#df4.to_csv(path_or_buf='results/HH/gene_correlation.csv')
df4

In [ ]:
proportional_significant_genes=df4.columns[df4.loc['correlation',:]>0.2]
inversely_significant_genes=df4.columns[df4.loc['correlation',:]<-0.2]
#print('proportionally significant genes:',proportional_significant_genes)
#print('inversely significant genes:',inversely_significant_genes)
df5=df4[proportional_significant_genes].T
df6=df4[inversely_significant_genes].T

In [ ]:
print('positive correlation')
df5

In [ ]:
print('negative correlation:')
df6

In [ ]:
umi_thresh = 1000
idx_kept, gene_labels_idx=filter_genes.filter_genes(counts_mat, gene_labels, 
                                       umi_threshold=umi_thresh, 
                                       exclude_prefix=['MT-', 'RPL', 'RPS'])

In [ ]:
# compute piecewise linear fit for restricted spots
pw_fit_dict=segmented_fit.pw_linear_fit(counts_mat_restrict, gaston_labels_restrict, gaston_isodepth_restrict,
                                        None, [],  idx_kept=idx_kept, umi_threshold=umi_thresh, isodepth_mult_factor=0.01,)
# for plotting
binning_output=binning_and_plotting.bin_data(counts_mat_restrict.T, gaston_labels_restrict, gaston_isodepth_restrict, 
                         None, gene_labels, idx_kept=idx_kept, num_bins=15, umi_threshold=umi_thresh)

In [ ]:
domain_colors=['plum', 'dodgerblue', 'cadetblue', '#F44E3F','orange']

domain_colors=['#16537e', '#93c47d', '#e99445', '#8ecae6','#6c4747']

discont_genes_layer=spatial_gene_classification.get_discont_genes(pw_fit_dict, binning_output,q=0.95)
cont_genes_layer=spatial_gene_classification.get_cont_genes(pw_fit_dict, binning_output,q=0.8)

In [ ]:
gene_name='COX7B'
print(f'gene {gene_name}: discontinuous jump after domain(s) {discont_genes_layer[gene_name]}') 
print(f'gene {gene_name}: continuous gradient in domain(s) {cont_genes_layer[gene_name]}')

# display log CPM (if you want to do CP500, set offset=500)
offset=10**6

binning_and_plotting.plot_gene_pwlinear(gene_name, pw_fit_dict, gaston_labels_restrict, gaston_isodepth_restrict, 
                                        binning_output, cell_type_list=None, pt_size=50, colors=domain_colors, 
                                        linear_fit=True, ticksize=15, figsize=(4,2.5), offset=offset, lw=3,
                                       domain_boundary_plotting=True)

In [ ]:
rotate=np.radians(-90)

binning_and_plotting.plot_gene_raw(gene_name, gene_labels_idx, counts_mat_restrict[:,idx_kept], S_restrict, vmin=5, 
                                   figsize=(6.11,4),s=10,rotate=rotate)
plt.title(f'{gene_name} Raw Expression')
binning_and_plotting.plot_gene_function(gene_name, S_restrict, pw_fit_dict, gaston_labels_restrict, gaston_isodepth_restrict, 
                                        binning_output, figsize=(6,4),rotate=rotate,s=10)
plt.title(f'{gene_name} GASTON Function')
plt.show()

In [ ]:
sc.pp.pca(adata)
sc.pp.neighbors(adata,n_neighbors=25,metric='euclidean')
sc.tl.leiden(adata,resolution=.2)
sc.tl.umap(adata)
cluster_spots=adata.obs["leiden"].tolist()
sc.pl.umap(adata, color='leiden', palette="plasma")
print('RNA-clusters:',np.unique(cluster_spots))

In [ ]:
np.shape(gaston_isodepth_restrict)
gaston_labels_restrict

In [ ]:
plt.scatter(np.arange(4468),gaston_isodepth_restrict, c=gaston_labels_restrict)

In [ ]:
plot_df=pd.DataFrame(data=counts_mat,columns=gene_labels)
plot_df=plot_df.assign(isodepth=gaston_isodepth)
plot_df=plot_df.assign(domains=gaston_labels)
plot_df=plot_df.assign(clusters=cluster_spots)
plot_df

In [ ]:
gene_name='TMSB10'
plt.subplots()
color = np.array(plot_df['clusters'], dtype=int)
plt.scatter(gaston_isodepth,plot_df[gene_name],c=color,cmap='plasma',s=4)
plt.ylabel('expression')

#spl = UnivariateSpline(gaston_isodepth,plot_df[gene_name])
#xs = np.linspace(-3, 3, 1000)
#plt.plot(xs, spl(xs), 'g', lw=3)

print(f'gene {gene_name}: discontinuous after domain(s) {discont_genes_layer[gene_name]}') 
print(f'gene {gene_name}: continuous in domain(s) {cont_genes_layer[gene_name]}')

# display log CPM (if you want to do CP500, set offset=500)
offset=10**6
binning_and_plotting.plot_gene_pwlinear(gene_name, pw_fit_dict, gaston_labels_restrict, gaston_isodepth_restrict, 
                                        binning_output, cell_type_list=None, pt_size=50, colors=domain_colors, 
                                        linear_fit=True, ticksize=15, figsize=(4.5,3), offset=offset, lw=3,
                                       domain_boundary_plotting=True)